# AutoML: Pycaret

In [ ]:
!pip install pycaret

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pycaret.classification import *

In [3]:
df=pd.read_csv('/kaggle/input/basic-of-kaggle-diabetes/train.csv')

In [4]:
df.head(5)

,ID,age,gender,polyuria,polydipsia,sudden_weight_loss,weakness,polyphagia,genital_thrush,visual_blurring,itching,irritability,delayed_healing,partial_paresis,muscle_stiffness,alopecia,obesity,class
0,0,41,Male,Yes,No,No,Yes,Yes,No,No,Yes,No,Yes,No,Yes,Yes,No,Positive
1,1,45,Male,No,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes,No,No,No,No,Positive
2,2,60,Male,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Positive
3,3,66,Male,Yes,Yes,Yes,Yes,No,No,Yes,Yes,Yes,No,Yes,Yes,No,No,Positive
4,4,67,Male,Yes,Yes,No,Yes,Yes,Yes,No,Yes,Yes,No,Yes,Yes,No,Yes,Positive


In [6]:
train_df = df.copy().drop(columns=["ID"])

In [7]:
s = setup(train_df, target = 'class', session_id = 123)

,Description,Value
0,Session id,123
1,Target,class
2,Target type,Binary
3,Target mapping,"Negative: 0, Positive: 1"
4,Original data shape,"(364, 17)"
5,Transformed data shape,"(364, 17)"
6,Transformed train set shape,"(254, 17)"
7,Transformed test set shape,"(110, 17)"
8,Ordinal features,15
9,Numeric features,1


In [8]:
exp = ClassificationExperiment()
exp.setup(train_df, target = 'class', session_id = 123)

,Description,Value
0,Session id,123
1,Target,class
2,Target type,Binary
3,Target mapping,"Negative: 0, Positive: 1"
4,Original data shape,"(364, 17)"
5,Transformed data shape,"(364, 17)"
6,Transformed train set shape,"(254, 17)"
7,Transformed test set shape,"(110, 17)"
8,Ordinal features,15
9,Numeric features,1


In [9]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.9725,0.9838,0.9725,0.9756,0.9723,0.9400,0.9435,0.1530
et,Extra Trees Classifier,0.9686,0.9919,0.9686,0.9707,0.9683,0.9311,0.9338,0.2760
rf,Random Forest Classifier,0.9646,0.9877,0.9646,0.9658,0.9644,0.9227,0.9243,0.2330
lightgbm,Light Gradient Boosting Machine,0.9608,0.9799,0.9608,0.9654,0.9607,0.9156,0.9202,0.4050
catboost,CatBoost Classifier,0.9569,0.9906,0.9569,0.9596,0.9569,0.9072,0.9098,1.9520
xgboost,Extreme Gradient Boosting,0.9489,0.9824,0.9489,0.9538,0.9490,0.8911,0.8956,0.1460
dt,Decision Tree Classifier,0.9452,0.9388,0.9452,0.9494,0.9448,0.8802,0.8853,0.1130
qda,Quadratic Discriminant Analysis,0.9406,0.9787,0.9406,0.9454,0.9394,0.8674,0.8745,0.1050
lr,Logistic Regression,0.9097,0.9709,0.9097,0.9164,0.9093,0.8061,0.8125,0.6140
ada,Ada Boost Classifier,0.9052,0.9660,0.9052,0.9136,0.9050,0.7967,0.8047,0.1570


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

In [10]:
exp.compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.9725,0.9838,0.9725,0.9756,0.9723,0.9400,0.9435,0.4480
et,Extra Trees Classifier,0.9686,0.9919,0.9686,0.9707,0.9683,0.9311,0.9338,0.4860
rf,Random Forest Classifier,0.9646,0.9877,0.9646,0.9658,0.9644,0.9227,0.9243,0.5720
lightgbm,Light Gradient Boosting Machine,0.9608,0.9799,0.9608,0.9654,0.9607,0.9156,0.9202,0.3100
catboost,CatBoost Classifier,0.9569,0.9906,0.9569,0.9596,0.9569,0.9072,0.9098,1.7700
xgboost,Extreme Gradient Boosting,0.9489,0.9824,0.9489,0.9538,0.9490,0.8911,0.8956,0.3310
dt,Decision Tree Classifier,0.9452,0.9388,0.9452,0.9494,0.9448,0.8802,0.8853,0.2880
qda,Quadratic Discriminant Analysis,0.9406,0.9787,0.9406,0.9454,0.9394,0.8674,0.8745,0.2840
lr,Logistic Regression,0.9097,0.9709,0.9097,0.9164,0.9093,0.8061,0.8125,0.3160
ada,Ada Boost Classifier,0.9052,0.9660,0.9052,0.9136,0.9050,0.7967,0.8047,0.4360


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='log_loss', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_samples_leaf=1,
                           min_samples_split=2, min_weight_fraction_leaf=0.0,
                           n_estimators=100, n_iter_no_change=None,
                           random_state=123, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [12]:
gbc = create_model('gbc')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,0.9231,0.9412,0.9231,0.9371,0.9245,0.8385,0.8497
2,0.9615,0.9625,0.9615,0.9638,0.9611,0.9172,0.9204
3,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,0.9600,0.9861,0.9600,0.9624,0.9594,0.9110,0.9147
5,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,0.9600,0.9722,0.9600,0.9640,0.9604,0.9153,0.9186
8,0.9200,0.9757,0.9200,0.9289,0.9174,0.8175,0.8315


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [16]:
tuned_gbc = tune_model(gbc, n_iter = 50,optimize = 'Accuracy',search_library = 'optuna')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,0.9231,0.9608,0.9231,0.9371,0.9245,0.8385,0.8497
2,0.9615,0.9875,0.9615,0.9638,0.9611,0.9172,0.9204
3,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
4,0.9600,0.9792,0.9600,0.9624,0.9594,0.9110,0.9147
5,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
6,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
7,0.9600,0.9931,0.9600,0.9640,0.9604,0.9153,0.9186
8,0.9200,0.9583,0.9200,0.9289,0.9174,0.8175,0.8315


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

[I 2024-01-28 05:42:50,623] Searching the best hyperparameters using 254 samples...
[I 2024-01-28 05:51:00,904] Finished hyperparameter search!


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [17]:
test=pd.read_csv('/kaggle/input/basic-of-kaggle-diabetes/test.csv')
test_df = test.copy().drop(columns=["ID"])


In [22]:
test

,ID,age,gender,polyuria,polydipsia,sudden_weight_loss,weakness,polyphagia,genital_thrush,visual_blurring,itching,irritability,delayed_healing,partial_paresis,muscle_stiffness,alopecia,obesity
0,0,57,Male,Yes,Yes,Yes,Yes,Yes,No,Yes,No,No,No,Yes,No,No,No
1,1,55,Female,Yes,Yes,Yes,Yes,No,No,Yes,No,No,No,Yes,No,No,Yes
2,2,30,Male,No,No,No,No,No,No,No,No,No,No,No,No,No,No
3,3,45,Female,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,No,No,No
4,4,47,Male,No,No,No,No,No,No,No,Yes,No,No,No,No,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,151,30,Male,No,No,No,No,No,No,No,No,No,No,No,No,No,No
152,152,38,Male,No,No,No,No,No,No,No,No,No,No,No,No,No,No
153,153,37,Male,No,No,No,No,No,No,No,No,No,No,No,No,No,No
154,154,50,Female,No,Yes,Yes,Yes,No,No,Yes,No,No,Yes,No,No,No,No


In [19]:
pred = predict_model(tuned_gbc, data=test_df)
pred.head()

,age,gender,polyuria,polydipsia,sudden_weight_loss,weakness,polyphagia,genital_thrush,visual_blurring,itching,irritability,delayed_healing,partial_paresis,muscle_stiffness,alopecia,obesity,prediction_label,prediction_score
0,57,Male,Yes,Yes,Yes,Yes,Yes,No,Yes,No,No,No,Yes,No,No,No,Positive,0.9989
1,55,Female,Yes,Yes,Yes,Yes,No,No,Yes,No,No,No,Yes,No,No,Yes,Positive,0.9978
2,30,Male,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Negative,0.9821
3,45,Female,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,No,No,No,Positive,0.9991
4,47,Male,No,No,No,No,No,No,No,Yes,No,No,No,No,Yes,No,Negative,0.9823


In [20]:
result = pd.DataFrame({
    "ID": test["ID"],
    "class": pred["prediction_label"]
})

In [21]:
result

,ID,class
0,0,Positive
1,1,Positive
2,2,Negative
3,3,Positive
4,4,Negative
...,...,...
151,151,Negative
152,152,Negative
153,153,Negative
154,154,Positive


In [23]:
result.to_csv("submission.csv", index=False)